In [2]:
import os 
os.getcwd()

'/Users/aleedom/cltk'

In [1]:
from cltk.corpus.utils.formatter import assemble_phi5_author_filepaths
from cltk.corpus.utils.formatter import phi5_plaintext_cleanup
from cltk.stem.latin.j_v import JVReplacer
from collections import Counter
from nltk.tokenize.punkt import PunktLanguageVars
import os
from cltk.lemmatize.latin.backoff import BackoffLatinLemmatizer
lemmatizer = BackoffLatinLemmatizer()

path = 'lexica/Lewis_Short_XML/lat.ls.perseus-eng1.xml'
path2 = 'lexica/Lewis_Short_XML/lat.ls.perseus-eng2.xml'

## Now that we've messed around a bit, why not create a filter list?

In [2]:
from collections import Counter
from cltk.corpus.utils.importer import CorpusImporter
corpus_importer = CorpusImporter('latin')
corpus_importer.list_corpora

['latin_text_perseus',
 'latin_treebank_perseus',
 'latin_text_latin_library',
 'phi5',
 'phi7',
 'latin_proper_names_cltk',
 'latin_models_cltk',
 'latin_pos_lemmata_cltk',
 'latin_treebank_index_thomisticus',
 'latin_lexica_perseus',
 'latin_training_set_sentence_cltk',
 'latin_word2vec_cltk',
 'latin_text_antique_digiliblt',
 'latin_text_corpus_grammaticorum_latinorum',
 'latin_text_poeti_ditalia',
 'latin_text_tesserae']

In [4]:
%load_ext autoreload
%autoreload 2
import steadman

ModuleNotFoundError: No module named 'docx'

In [5]:
from docx import Document

ModuleNotFoundError: No module named 'docx'

In [7]:
from cltk.corpus.readers import get_corpus_reader
corpus_importer.import_corpus('latin_text_perseus')
reader = get_corpus_reader(language='latin', corpus_name='latin_text_latin_library')


## We choose a sample document...

In [556]:
punc

['.', ',', ';', '"', "'", '-que', '-ne', '-ve']

In [355]:
S = CorpusStoplist()

In [361]:
stops = S.build_stoplist(str([[x] for x in reader.docs()]), size=500)

stops = list(set([lemmatizer.lemmatize([x])[0][1] for x in stops]))

stops

['accurro',
 'agito',
 'addo',
 'citer',
 'aethiopia',
 'adeo1',
 'admisceo',
 'adprehendo',
 'dum',
 'absolvo',
 'assuesco',
 'agilis',
 'homo',
 'affluo',
 'sum',
 'haud',
 'adolesco',
 'gradus',
 'caesar',
 'aestimo',
 'aereus1',
 'aevum',
 'aliter',
 'doceo',
 'administro',
 'eques',
 'acerbus',
 'communio1',
 'aliquoties',
 'adfabilitas',
 'alius',
 'ad-luo1',
 'apodemio',
 'ad-sequor',
 'actito',
 'adnecto',
 'antiochiam',
 'fortuna',
 'ad-scisco',
 'gallus',
 'audacia',
 'cognosco',
 'amplus',
 'cogito',
 'contemplo',
 'ab',
 'bene',
 'abdo',
 'gravis',
 'adeo',
 'constantina',
 'defero',
 'incertus',
 'imperator',
 'deinde',
 'ideo',
 'armenia',
 'duco',
 'epigonus',
 'ambo',
 'adduco',
 'adverso',
 'collaticius',
 'confine',
 'ascitus2',
 'dignitas',
 'adigentibus',
 'consortes',
 'administratio',
 'alter',
 'aditibus',
 'affatim',
 'aurigaris',
 'aboleo',
 'adepturi',
 'cunctus',
 'aeger',
 'crus',
 'fero',
 'aedifico',
 'ex',
 'carnifex',
 'admitto',
 'cum2',
 'adimo',
 'adu

In [9]:
reader._fileids = ['ammianus/14.txt'] # ammianus book 14

In [27]:
paras = list(reader.paras())
numbers = [str(n) for n in range(11)]

In [33]:
paragraph = []
for subsection in paras[3]:
    if subsection[0].strip('.') not in numbers: 
        paragraph.append(" ".join(subsection))

## ... And clean it up a bit 

In [52]:
for p in paras: 
    if len(p) >1: 
        for chunk in p: 
            if chunk([0]) not in numbers: 
                paragraph.append(" ".join(subsection))

1
1
1
23
30
9
10
21
45
34
30
15
27
58


In [470]:


def clean_paragraph(ls):
    out = f""
    for i in range(len(ls) - 1):  
        if ls[i + 1] in punc: 
            out += ''.join([ls[i], ls[i + 1].strip('-')]) + ' '
        elif ls[i] not in punc:
            out += f'{ls[i]} '
            
        else: 
            pass
    return out.rstrip(' ')

clean_paragraph(paras[6][1])

'Saraceni tamen nec amici nobis umquam nec hostes optandi, ultro citroque discursantes quicquid inveniri poterat momento temporis parvi vastabant milvorum rapacium similes, qui si praedam dispexerint celsius, volatu rapiunt celeri, aut nisi impetraverint, non inmorantur.'

In [98]:
def clean_paragraph(paragraphs: "list") -> 'string': 
    """Cleans Latin Library paragraph text. 
    :param paragraphs: list of paragraphs from CLTK reader.paras() method. 
    
    Works, kind of
    """
    numbers = [str(n) for n in range(11)]
    punc = ['.', ',', ';', '"', "'", '-que', '-ne']
    final = []
    for p in paragraphs: 
        out = ""
        if len(p) > 1: # filter out section headings
            for chunk in p:
                for i in range(len(chunk) - 1):  # doesn't break because puncutation should always end a section
                    if chunk[i + 1] in punc: 
                        out += ''.join([chunk[i], chunk[i + 1]]).strip('-') + ' '
                    elif chunk[i] not in punc:
                        out += f'{chunk[i]} '
                    else: 
                        pass
        final.append(out)
    return "\n".join(final)

In [484]:
def parse_paragraph(paragraph: 'str') -> "str":
    """Function to take paragraph of a Latin text and return a dictionary including definitions (but not citations). 
    The goal is to use this function to create short entries for a paragraph of a text. We can then use the paragraphs to build our Pharr formatted document. 
    
    :param paragraph: paragraph of parsed text
    """
    
    in_list = paragraph.split(' ')
    in_list = [_ for _ in in_list if _ not in STOPS_LIST]
    out_str = '' 
    path = 'lexica/Lewis_Short_XML/lat.ls.perseus-eng1.xml'
    tree = ET.parse(path)
    entries = tree.xpath('//entryFree')
    endings = ''
    gender = ""
    out_list = []
    
    for word in in_list: 
        lemma = lemmatizer.lemmatize([word])[0][1]
        
        for entry in entries:
            senses = []
            if entry.get('key') == lemma:
                if entry.find('itype') is not None: 
                    endings = f'{lemma} {entry.find("itype").text}'
                if entry.find('gen') is not None: 
                    gender = entry.find('gen').text
                for sense in entry.findall('sense')[:4]:
                    # print(sense.get('level'))
                    if sense.get("level") in ['1', '2']:
                        for tr in sense.findall('hi')[1:3]:
                            senses.append(tr.text)
#                 print(senses)
                if endings != '':
                    out_string = f"""{endings} {gender}: {'; '.join(senses).strip('., ')}"""
                else:
                    out_string = f'{lemma} {gender}: {"; ".join(senses).strip("., ")}'
                if senses == []:
                    pass 
                else:
                    out_list.append(out_string)
    return '\n'.join(out_list)

In [313]:
test = parse_paragraph(' '.join(paras[3][4]))

100%|██████████| 67/67 [00:02<00:00, 25.98it/s]


In [314]:
br = "—————————————————————————————————————————————————————————————"

In [318]:
p = clean_paragraph(paras[3][4])
print(f'{p} \n {br} \n {parse_paragraph(p)}')

100%|██████████| 58/58 [00:02<00:00, 27.17it/s]

Cuius acerbitati uxor grave accesserat incentivum, germanitate Augusti turgida supra modum, quam Hannibaliano regi fratris filio antehac Constantinus iunxerat pater, Megaera quaedam mortalis, inflammatrix saevientis adsidua, humani cruoris avida nihil mitius quam maritus; qui paulatim eruditiores facti processu temporis ad nocendum per clandestinos versutosque rumigerulos conpertis leviter addere quaedam male suetos falsa et placentia sibi discentes, adfectati regni vel artium nefandarum calumnias insontibus adfligebant. 
 ————————————————————————————————————————————————————————————— 
 acerbitas ātis f.: sourness; harshness; severity; rigor
uxor ōris f.: spouse; consort
gravis e f.: weighty; ponderous
accedo cessi, cessum, 3 f.: to go; come to; in; adverbs
germanitas ātis f.: brotherhood; sisterhood
turgidus a, um f.: inflated; distended; turgid
turgidus a, um f.: on the upper side; on the top; stretched over them; Comp
frater tris m.: Do.; To
filius ii m.: a son
antehac like antidea f

## Now test Docx

In [11]:
!pip install python-docx

In [485]:
from docx import Document
from docx.oxml import OxmlElement
from docx.oxml.ns import qn

doc = Document()

In [448]:
len(doc.sections)

1

In [440]:
doc.add_paragraph(p)

doc.add_paragraph(br)


doc.add_section(0)

section = doc.sections[1]

In [441]:
sectPr = section._sectPr
cols = sectPr.xpath('./w:cols')[0]
cols.set(qn('w:num'),'2')

In [442]:
doc.add_paragraph(parse_paragraph(p))

100%|██████████| 58/58 [00:01<00:00, 30.16it/s]


In [443]:
doc.add_page_break()

In [444]:
doc.add_section(0)
section = doc.sections[-1]
sectPr = section._sectPr
cols = sectPr.xpath('./w:cols')[0]
cols.set(qn('w:num'),'1')
doc.add_paragraph(paras[4][3])

In [445]:
doc.save('demo.docx')

In [325]:
doc.add_paragraph(br)

In [326]:
doc.add_paragraph(parse_paragraph(p))

100%|██████████| 58/58 [00:01<00:00, 31.95it/s]


In [328]:
br = '———————————————————————————————————————'

In [461]:
paras[3][5]

['3.',
 'eminuit',
 'autem',
 'inter',
 'humilia',
 'supergressa',
 'iam',
 'impotentia',
 'fines',
 'mediocrium',
 'delictorum',
 'nefanda',
 'Clematii',
 'cuiusdam',
 'Alexandrini',
 'nobilis',
 'mors',
 'repentina',
 ';',
 'cuius',
 'socrus',
 'cum',
 'misceri',
 'sibi',
 'generum',
 ',',
 'flagrans',
 'eius',
 'amore',
 ',',
 'non',
 'impetraret',
 ',',
 'ut',
 'ferebatur',
 ',',
 'per',
 'palatii',
 'pseudothyrum',
 'introducta',
 ',',
 'oblato',
 'pretioso',
 'reginae',
 'monili',
 'id',
 'adsecuta',
 'est',
 ',',
 'ut',
 'ad',
 'Honoratum',
 'tum',
 'comitem',
 'orientis',
 'formula',
 'missa',
 'letali',
 'omnino',
 'scelere',
 'nullo',
 'contactus',
 'idem',
 'Clematius',
 'nec',
 'hiscere',
 'nec',
 'loqui',
 'permissus',
 'occideretur',
 '.']

In [472]:

for graf in tqdm(paras): 
    print(len(graf))
    if len(graf) > 1: 
        for g in graf:
            p = clean_paragraph(g)


100%|██████████| 14/14 [00:00<00:00, 294.57it/s]

1
1
1
23
I 1.
Post emensos insuperabilis expeditionis eventus languentibus partium animis, quas periculorum varietas fregerat et laborum, nondum tubarum cessante clangore vel milite locato per stationes hibernas, fortunae saevientis procellae tempestates alias rebus infudere communibus per multa illa et dira facinora Caesaris Galli, qui ex squalore imo miseriarum in aetatis adultae primitiis ad principale culmen insperato saltu provectus ultra terminos potestatis delatae procurrens asperitate nimia cuncta foedabat.
propinquitate enim regiae stirpis gentilitateque etiam tum Constantini nominis efferebatur in fastus, si plus valuisset, ausurus hostilia in auctorem suae felicitatis, ut videbatur.
2.
Cuius acerbitati uxor grave accesserat incentivum, germanitate Augusti turgida supra modum, quam Hannibaliano regi fratris filio antehac Constantinus iunxerat pater, Megaera quaedam mortalis, inflammatrix saevientis adsidua, humani cruoris avida nihil mitius quam maritus; qui paulatim eruditio

In [480]:
from tqdm.autonotebook import tqdm

/Users/aleedom/cltk/venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  """Entry point for launching an IPython kernel.


In [4]:

from tqdm import tqdm

# blah blah your code errored



In [531]:
p = clean_paragraph(paras[3][7])

In [533]:
len(p.split(' '))

21

In [555]:
lemmatizer.lemmatize(['rumigerabatur'])

[('rumigerabatur', 'rumigerabatur')]

In [546]:
for section in paras[:4]: 
    collector = f''
    while len(collector.split(' ')) < 100:
        for subsection in section: 
            p = clean_paragraph(subsection) + " "
            collector+=(p)
    p = collector
    print(p)

Ammianus : Liber Ammianus : Liber Ammianus : Liber Ammianus : Liber Ammianus : Liber Ammianus : Liber Ammianus : Liber Ammianus : Liber Ammianus : Liber Ammianus : Liber Ammianus : Liber Ammianus : Liber Ammianus : Liber Ammianus : Liber Ammianus : Liber Ammianus : Liber Ammianus : Liber Ammianus : Liber Ammianus : Liber Ammianus : Liber Ammianus : Liber Ammianus : Liber Ammianus : Liber Ammianus : Liber Ammianus : Liber Ammianus : Liber Ammianus : Liber Ammianus : Liber Ammianus : Liber Ammianus : Liber Ammianus : Liber Ammianus : Liber Ammianus : Liber 
AMMIANI MARCELLINI HISTORIAE LIBER AMMIANI MARCELLINI HISTORIAE LIBER AMMIANI MARCELLINI HISTORIAE LIBER AMMIANI MARCELLINI HISTORIAE LIBER AMMIANI MARCELLINI HISTORIAE LIBER AMMIANI MARCELLINI HISTORIAE LIBER AMMIANI MARCELLINI HISTORIAE LIBER AMMIANI MARCELLINI HISTORIAE LIBER AMMIANI MARCELLINI HISTORIAE LIBER AMMIANI MARCELLINI HISTORIAE LIBER AMMIANI MARCELLINI HISTORIAE LIBER AMMIANI MARCELLINI HISTORIAE LIBER AMMIANI MARCELLINI

In [488]:
doc = Document()
for graf in tqdm(paras[:10]):  
    for g in graf:
        p = clean_paragraph(g)
            if len(p.split(' ')) < 60:
                # page will fit more, so stack an additional paragraph 
        doc.add_paragraph(p)

        doc.add_paragraph(br)
        doc.add_section(0)
        section = doc.sections[-1]

        sectPr = section._sectPr
        cols = sectPr.xpath('./w:cols')[0]
        cols.set(qn('w:num'),'2')
        doc.add_paragraph(parse_paragraph(p))

        doc.add_section(0)
        section = doc.sections[-1]

        sectPr = section._sectPr
        cols = sectPr.xpath('./w:cols')[0]
        cols.set(qn('w:num'),'1')
        doc.add_page_break()


doc.save('test.docx')

        

  0%|          | 0/10 [00:00<?, ?it/s]

16


 10%|█         | 1/10 [00:03<00:27,  3.07s/it]

34


 20%|██        | 2/10 [00:06<00:24,  3.10s/it]

30


 30%|███       | 3/10 [00:09<00:21,  3.11s/it]

4
506
181
2
526
471
2
159
259
294
173
393


 30%|███       | 3/10 [00:55<02:08, 18.35s/it]


KeyboardInterrupt: 

In [ ]:
def make_docx(text: 'list') -> 'docx': 
    "makes a docx"
    

## Now fix definitions

In [177]:
def lookup_word(word: "str") -> "dict": 
    "parses the LS for word information. Limits the returned definitions to two senses for brevity"
    
    path = 'lexica/Lewis_Short_XML/lat.ls.perseus-eng1.xml'
    tree = ET.parse(path)
    entries = tree.xpath('//entryFree')
    
    d = {}
    lemma = lemmatizer.lemmatize([word])[0][1]
    count = 0

    for entry in tqdm(entries):      
        
        senses = [] # definitions
        endings = [] # word form 
        # quotes = [] # no quotes for the text portion 
        gender = '' 
        d[lemma] = {} 
        
        if entry.get('key') == lemma: 
            print('match')
            if entry.find('itype') is not None: 
                print(f'{lemma} {entry.find("itype").text}') 
            if entry.find('gen') is not None: 
                print(entry.find('gen').text)
            for sense in entry.findall('sense')[:4]:
                print(sense.get('level'))
                if sense.get("level") in ['1', '2']:
                    for tr in sense.findall('hi')[:2]:
                        print(tr.text)

In [178]:
lookup_word('abdico')

100%|██████████| 51596/51596 [00:00<00:00, 759456.15it/s]

match
abdico1 āvi, ātum, 1
1
to indicate, announce
not belonging
1
to deny, disown, refuse, reject
inf.
1
2
to renounce
to disinherit


In [304]:
words = list(set(reader.words()))
words = [w.lower() for w in words if w != '']
dictionary = {}
# lemmatize

final = []
count = 0
for w in tqdm(words): 
    root = lemmatizer.lemmatize([w])
    if len(root) > 1:
        print('long entry')
    dictionary[root[0][1]] = {} 
    dictionary[root[0][1]]['token']  = root[0][0]
    dictionary[root[0][1]]['lemma'] = root[0][1]


test = random.sample(list(dictionary.keys()), k = 10)

In [317]:
count = 0
tree = ET.parse(path)
entries = tree.xpath('//entryFree')
d = {}
for word in tqdm(dictionary.keys()): 
    senses = []
    endings = []
    quotes = []
    gender = ''
    for entry in entries: 
        d[word] = {}
        if entry.get('type') != 'main': 
            pass
        if entry.get('key') == word: 
            if entry.find('itype') is not None:
                endings.append(entry.find('itype').text)
            if entry.find('gen') is not None: 
                gender = entry.find('gen').text
            for sense in entry.findall('sense'):
                count = 0
#                 print(sense.get('n'), sense.get('level'))
                for hi in sense.findall('hi'):
                    if hi.text is not None:
                        senses.append(f'Sense {sense.get("n")}: {hi.text}')
                        count += 1
                    if count == 5: 
                        continue
                for cit in sense.findall('cit'):
                    count = 0
                    for quote in cit.findall('quote'):
                        if quote.text is not None:
                            quotes.append(f'Sense {sense.get("n")}: {quote.text}') 
        d[word]['senses']= "<br>".join(senses)
        d[word]['endings'] = ' -'.join(endings)
        d[word]['gender'] = gender
        d[word]['sentences'] = '<br>'.join(quotes)

100%|██████████| 8167/8167 [14:11<00:00,  9.60it/s]


In [318]:
eligible = count_df[(count_df['cumsum'] > .8) & (count_df['token_count'] >=2)]

elig = list(eligible.lemma)

delete = [k for k in d.keys() if k not in elig]
    



## Now print layout